In [ ]:
!pip install cython

In [1]:
import penaltyblog as pb
import pandas as pd

In [16]:
df = pd.concat([
    pb.scrapers.FootballData("NLD Eredivisie", "2017-2018").get_fixtures(),
    pb.scrapers.FootballData("NLD Eredivisie", "2018-2019").get_fixtures(),
    pb.scrapers.FootballData("NLD Eredivisie", "2019-2020").get_fixtures(),
    pb.scrapers.FootballData("NLD Eredivisie", "2020-2021").get_fixtures(),
    pb.scrapers.FootballData("NLD Eredivisie", "2021-2022").get_fixtures(),
    pb.scrapers.FootballData("NLD Eredivisie", "2022-2023").get_fixtures(),
    pb.scrapers.FootballData("NLD Eredivisie", "2023-2024").get_fixtures(),
    pb.scrapers.FootballData("NLD Eredivisie", "2024-2025").get_fixtures(),
])

df = df[df["date"] < "2025-03-01"]
df.shape

df = df.sort_values('date').set_index('date', drop=False)
ftr_map = {"H": 0, "D": 1, "A": 2}
df['ftr_numeric'] = df['ftr'].map(ftr_map)

train = df[df["date"] > "2010-01-01"]
train = train.reset_index(drop=True)
train.shape


/Users/martin/repos/penaltyblog/penaltyblog/scrapers/footballdata.py:68: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)


(2276, 159)

In [17]:
import cython

In [18]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [52]:
%%cython
# cython: boundscheck=False, wraparound=False, cdivision=True, nonecheck=False, initializedcheck=False, language_level=3
# Adding optimization flags (if supported by your compiler) can be done in setup.py (see below)

import numpy as np
cimport numpy as np
from libc.math cimport exp, log, lgamma
from cython.parallel import prange


def compute_llk_cython(np.int64_t[:] goals_home,
                       np.int64_t[:] goals_away,
                       np.float64_t[:] weights,
                       np.int64_t[:] home_indices,
                       np.int64_t[:] away_indices,
                       np.float64_t[:] attack,
                       np.float64_t[:] defence,
                       double hfa) -> double:
    cdef Py_ssize_t i, n = goals_home.shape[0]
    cdef double total_llk = 0.0
    cdef double lambda_home, lambda_away, llk_home, llk_away
    cdef int home_idx, away_idx, k_home, k_away

    # Main loop: optimized for performance
    with nogil:
        for i in prange(n):
            home_idx = home_indices[i]
            away_idx = away_indices[i]
            
            # Compute expected goals
            lambda_home = exp(hfa + attack[home_idx] + defence[away_idx])
            lambda_away = exp(attack[away_idx] + defence[home_idx])
            
            # Retrieve observed goals
            k_home = goals_home[i]
            k_away = goals_away[i]
            
            # Compute Poisson log-likelihood contributions
            llk_home = -lambda_home + k_home * log(lambda_home) - lgamma(k_home + 1)
            llk_away = -lambda_away + k_away * log(lambda_away) - lgamma(k_away + 1)
            
            total_llk += (llk_home + llk_away) * weights[i]
        
    return total_llk


In [20]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import poisson
import pandas as pd
import warnings

# Assume compute_llk_cython is compiled and available from the Cython cell above

class PoissonGoalsModel:
    def __init__(self, goals_home, goals_away, teams_home, teams_away, weights=1):
        self.fixtures = pd.DataFrame([goals_home, goals_away, teams_home, teams_away]).T
        self.fixtures.columns = ["goals_home", "goals_away", "team_home", "team_away"]
        self.fixtures["goals_home"] = self.fixtures["goals_home"].astype(int)
        self.fixtures["goals_away"] = self.fixtures["goals_away"].astype(int)
        self.fixtures["weights"] = weights

        self.teams = np.sort(np.unique(np.concatenate([teams_home, teams_away])))
        self.n_teams = len(self.teams)

        self._params = np.concatenate(
            (
                [1] * self.n_teams,
                [-1] * self.n_teams,
                [0.5],  # home advantage
            )
        )

        self._res = None
        self.loglikelihood = None
        self.aic = None
        self.n_params = None
        self.fitted = False

    @staticmethod
    def _fit(params, fixtures, teams):
        """
        Internal method using Cython for speed.
        """
        n_teams = len(teams)
        attack = params[:n_teams]
        defence = params[n_teams:2*n_teams]
        hfa = params[-1]

        # Create mapping from team names to indices
        team_to_idx = {team: i for i, team in enumerate(teams)}

        # Convert DataFrame columns to numpy arrays
        goals_home = fixtures["goals_home"].to_numpy().astype(np.int64)
        goals_away = fixtures["goals_away"].to_numpy().astype(np.int64)
        weights = fixtures["weights"].to_numpy().astype(np.double)
        team_home = fixtures["team_home"].to_numpy()
        team_away = fixtures["team_away"].to_numpy()

        # Map team names to indices for home and away teams
        home_indices = np.array([team_to_idx[team] for team in team_home], dtype=np.int64)
        away_indices = np.array([team_to_idx[team] for team in team_away], dtype=np.int64)

        # Convert attack and defence parameters to numpy arrays of type double
        attack = np.asarray(attack, dtype=np.double)
        defence = np.asarray(defence, dtype=np.double)

        # Call the Cython function for likelihood computation
        total_llk = compute_llk_cython(goals_home, goals_away, weights, home_indices, away_indices, attack, defence, hfa)
        return -total_llk

    def fit(self):
        options = {"maxiter": 100, "disp": False}
        constraints = [{"type": "eq", "fun": lambda x: sum(x[: self.n_teams]) - self.n_teams}]
        bounds = [(-3, 3)] * self.n_teams + [(-3, 3)] * self.n_teams + [(0, 3)]

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            self._res = minimize(
                self._fit,
                self._params,
                args=(self.fixtures, self.teams),
                constraints=constraints,
                bounds=bounds,
                options=options,
            )

        self._params = self._res["x"]
        self.n_params = len(self._params)
        self.loglikelihood = self._res["fun"] * -1
        self.aic = -2 * (self.loglikelihood) + 2 * self.n_params
        self.fitted = True

    # ... (other methods remain unchanged)


In [21]:
%%timeit
# Go version
clf = pb.models.PoissonGoalsModel(
    train["goals_home"],
    train["goals_away"],
    train["team_home"],
    train["team_away"],
)
clf.fit()

675 ms ± 39.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
clf = pb.models.PoissonGoalsModel(
    train["goals_home"],
    train["goals_away"],
    train["team_home"],
    train["team_away"],
)
clf.fit()
clf.loglikelihood

-6894.355718264861

In [ ]:
%%timeit
# Cython version
clf = PoissonGoalsModel(
    train["goals_home"],
    train["goals_away"],
    train["team_home"],
    train["team_away"],
)
clf.fit()

1.49 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
clf = PoissonGoalsModel(
    train["goals_home"],
    train["goals_away"],
    train["team_home"],
    train["team_away"],
)
clf.fit()
clf.loglikelihood

-6894.3557251866605

In [46]:
%%cython
# cython: boundscheck=False, wraparound=False, cdivision=True, language_level=3
import numpy as np
cimport numpy as np
from libc.math cimport exp, log, lgamma, isnan, isinf
from cython.parallel import prange

# Helper function for the Negative Binomial log-PMF:
cdef double negBinomLogPMF(int k, double dispersion, double p) nogil:
    """
    Computes the Negative Binomial log-PMF using the formula:
      logPMF = lgamma(k + dispersion) - lgamma(k + 1) - lgamma(dispersion)
               + dispersion * log(p) + k * log(1 - p)
    """
    return lgamma(k + dispersion) - lgamma(k + 1) - lgamma(dispersion) + dispersion * log(p) + k * log(1 - p)

cpdef double compute_negative_binomial_loss(np.float64_t[:] params,
                                   int nTeams,
                                   np.int32_t[:] homeIdx,
                                   np.int32_t[:] awayIdx,
                                   np.int32_t[:] goalsHome,
                                   np.int32_t[:] goalsAway,
                                   np.float64_t[:] weights,
                                   int nMatches):
    """
    Compute the negative log-likelihood for the Negative Binomial model.
    
    Parameters:
      params: Array of parameters of length (2*nTeams + 2), where the first nTeams
              elements are attack parameters, the next nTeams are defence parameters,
              followed by home advantage and dispersion.
      nTeams: Number of teams.
      homeIdx, awayIdx: Integer arrays (length nMatches) of team indices.
      goalsHome, goalsAway: Integer arrays (length nMatches) of goals scored.
      weights: A float array (length nMatches) of weights.
      nMatches: Number of matches.
      
    Returns:
      The negative log-likelihood.
    """
    cdef int i, idxHome, idxAway, kHome, kAway
    cdef double logLikelihood = 0.0
    cdef double homeAdvantage = params[2 * nTeams]
    cdef double dispersion = params[2 * nTeams + 1]
    # Ensure dispersion is at least 1e-5
    if dispersion < 1e-5:
        dispersion = 1e-5

    cdef double lambdaHome, lambdaAway, pHome, pAway
    cdef double logP_home, logP_away

    # Loop over each match
    for i in prange(nMatches, nogil=True):
        idxHome = homeIdx[i]
        idxAway = awayIdx[i]
        # Attack parameters: first nTeams elements; Defence: next nTeams elements
        lambdaHome = exp(homeAdvantage + params[idxHome] + params[nTeams + idxAway])
        lambdaAway = exp(params[idxAway] + params[nTeams + idxHome])
        pHome = dispersion / (dispersion + lambdaHome)
        pAway = dispersion / (dispersion + lambdaAway)
        kHome = goalsHome[i]
        kAway = goalsAway[i]
        logP_home = negBinomLogPMF(kHome, dispersion, pHome)
        logP_away = negBinomLogPMF(kAway, dispersion, pAway)
        # If any computed log likelihood is NaN or Inf, return a very large number
        if isnan(logP_home) or isinf(logP_home) or isnan(logP_away) or isinf(logP_away):
            return 1e308  # Use a large number to indicate failure
        logLikelihood += (logP_home + logP_away) * weights[i]

    return -logLikelihood


Content of stderr:
In file included from /Users/martin/.cache/ipython/cython/_cython_magic_fd007d85e40caf83f4ce850c92379f7373893c60.c:1254:
In file included from /Users/martin/repos/penaltyblog/venv/lib/python3.13/site-packages/numpy/_core/include/numpy/arrayobject.h:5:
In file included from /Users/martin/repos/penaltyblog/venv/lib/python3.13/site-packages/numpy/_core/include/numpy/ndarrayobject.h:12:
In file included from /Users/martin/repos/penaltyblog/venv/lib/python3.13/site-packages/numpy/_core/include/numpy/ndarraytypes.h:1909:
/Users/martin/repos/penaltyblog/venv/lib/python3.13/site-packages/numpy/_core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^
/Users/martin/.cache/ipython/cython/_cython_magic_fd007d85e40caf83f4ce850c92379f7373893c60.c:24220:13: warning: code will never b

In [47]:
import ctypes
import warnings

import numpy as np
from scipy.optimize import minimize

from penaltyblog.golib.loss import negative_binomial_loss_function
from penaltyblog.golib.probabilities import (
    compute_negative_binomial_probabilities,
)
from penaltyblog.models.base_model import BaseGoalsModel
from penaltyblog.models.custom_types import (
    GoalInput,
    ParamsOutput,
    TeamInput,
    WeightInput,
)
from penaltyblog.models.football_probability_grid import (
    FootballProbabilityGrid,
)


class NegativeBinomialGoalModel(BaseGoalsModel):
    """
    Negative Binomial model for predicting outcomes of football (soccer) matches
    handling overdispersion in goal data.

    Methods
    -------
    fit()
        fits a Negative Binomial model to the data to calculate the team strengths.
        Must be called before the model can be used to predict game outcomes

    predict(home_team, away_team, max_goals=10)
        predicts the probability of each scoreline for a given home and away team

    get_params()
        provides access to the model's fitted parameters
    """

    def __init__(
        self,
        goals_home: GoalInput,
        goals_away: GoalInput,
        teams_home: TeamInput,
        teams_away: TeamInput,
        weights: WeightInput = None,
    ):
        """
        Initialises the NegativeBinomialGoalModel class.

        Parameters
        ----------
        goals_home : array_like
            The number of goals scored by the home team
        goals_away : array_like
            The number of goals scored by the away team
        teams_home : array_like
            The names of the home teams
        teams_away : array_like
            The names of the away teams
        weights : array_like, optional
            The weights of the matches, by default None
        """
        super().__init__(goals_home, goals_away, teams_home, teams_away, weights)

        self._params = np.concatenate(
            ([1] * self.n_teams, [-1] * self.n_teams, [0.25], [0.1])
        )  # Home advantage and dispersion parameter

    def __repr__(self):
        lines = ["Module: Penaltyblog", "", "Model: Negative Binomial", ""]

        if not self.fitted:
            lines.append("Status: Model not fitted")
            return "\n".join(lines)

        lines.extend(
            [
                f"Number of parameters: {self.n_params}",
                f"Log Likelihood: {round(self.loglikelihood, 3)}",
                f"AIC: {round(self.aic, 3)}",
                "",
                "{0: <20} {1:<20} {2:<20}".format("Team", "Attack", "Defence"),
                "-" * 60,
            ]
        )

        for idx, team in enumerate(self.teams):
            lines.append(
                "{0: <20} {1:<20} {2:<20}".format(
                    team,
                    round(self._params[idx], 3),
                    round(self._params[idx + self.n_teams], 3),
                )
            )

        lines.extend(
            [
                "-" * 60,
                f"Home Advantage: {round(self._params[-2], 3)}",
                f"Dispersion: {round(self._params[-1], 3)}",
            ]
        )

        return "\n".join(lines)

    def _loss_function(self, params: np.ndarray) -> float:
        """
        Calculates the negative log-likelihood of the Negative Binomial model.

        Parameters
        ----------
        params : array_like
            The parameters of the model
        data : dict
            The data used to fit the model
        n_teams : int
            The number of teams in the league

        Returns
        -------
        float
            The negative log-likelihood of the Negative Binomial model
        """
        params = np.ascontiguousarray(params, dtype=np.float64)
        params_ctypes = params.ctypes.data_as(ctypes.POINTER(ctypes.c_double))     

        return compute_negative_binomial_loss(
            params,
            self.n_teams,
            self.home_idx,
            self.away_idx,
            self.goals_home,
            self.goals_away,
            self.weights,
            len(self.goals_home),
        )

    def fit(self):
        """
        Fits the Negative Binomial model to the data.
        """
        options = {"maxiter": 2500, "disp": False}
        bounds = [(-2, 2)] * self.n_teams * 2 + [(-4, 4), (1e-5, 1000)]
        constraints = [
            {"type": "eq", "fun": lambda x: sum(x[: self.n_teams]) - self.n_teams}
        ]

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            self._res = minimize(
                self._loss_function,
                self._params,
                bounds=bounds,
                constraints=constraints,
                options=options,
            )

        if not self._res.success:
            raise ValueError(f"Optimization failed with message: {self._res.message}")

        self._params = self._res.x
        self.n_params = len(self._params)
        self.loglikelihood = -self._res.fun
        self.aic = -2 * self.loglikelihood + 2 * self.n_params
        self.fitted = True

    def get_params(self):
        pass

    def predict():
        pass
    
    

    


In [48]:
%%timeit
clf = pb.models.NegativeBinomialGoalModel(
    train["goals_home"],
    train["goals_away"],
    train["team_home"],
    train["team_away"],
)
clf.fit()

1.57 s ± 37.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
clf = pb.models.NegativeBinomialGoalModel(
    train["goals_home"],
    train["goals_away"],
    train["team_home"],
    train["team_away"],
)
clf.fit()
clf.loglikelihood

-6894.8293946936665

In [50]:
%%timeit
# Cython version
clf = NegativeBinomialGoalModel(
    train["goals_home"],
    train["goals_away"],
    train["team_home"],
    train["team_away"],
)
clf.fit()

998 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
clf = NegativeBinomialGoalModel(
    train["goals_home"],
    train["goals_away"],
    train["team_home"],
    train["team_away"],
)
clf.fit()
clf.loglikelihood

-6894.954007748044